Import potrzebnych bibliotek

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re, string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
import csv

Wczytanie danych

In [2]:
train = pd.read_csv("train.csv", sep=",")
test = pd.read_csv("test.csv", sep=",")

reviews = train['Reviews'].astype(str)
ratings = train['Rating']
test_data = test['Reviews'].astype(str)

Przygotowanie danych - usunięcie znaków, cyfr, zmiana na małe litery

In [3]:
for i, review2 in reviews.items():
    reviews[i] = reviews[i].replace('_','')
    reviews[i] = re.sub(r'http.?://[^\s]+[\s]?', '', reviews[i])
    reviews[i] = re.sub(r'@\w+', '', reviews[i])
    reviews[i] = ' '.join([j for j in reviews[i].split() if not any(c.isdigit() for c in j)])
    table = str.maketrans({key: None for key in string.punctuation})
    reviews[i] = reviews[i].translate(table)
    reviews[i] = reviews[i].lower()

In [4]:
for i, review3 in test_data.items():
    test_data[i] = test_data[i].replace('_','')
    test_data[i] = re.sub(r'http.?://[^\s]+[\s]?', '', test_data[i])
    test_data[i] = re.sub(r'@\w+', '', test_data[i])
    test_data[i] = ' '.join([j for j in test_data[i].split() if not any(c.isdigit() for c in j)])
    table = str.maketrans({key: None for key in string.punctuation})
    test_data[i] = test_data[i].translate(table)
    test_data[i] = test_data[i].lower()

Przygotowanie danych - usunięcie stop words nie zawierających negacji (ostatecznie miało to negatywny wpływ na wynik wiec nie było wykorzystywane w dalszych próbach)

In [5]:
#stopWords = set(stopwords.words('english'))
#negative = {"can't", "cannot", "couldn't", "didn't", "doesn't", "don't", "down", "few", "hadn't", "hasn't", "haven't",
#            "isn't", "mustn't", "no", "nor", "not", "off", "out", "shouldn't", "won't", "wouldn't", "needn't", "weren't"}
#stopWords = stopWords - negative

In [6]:
#for i, review in test_data.items():
 #   test_data[i] = ' '.join([j for j in review.split() if j not in stopWords])

In [7]:
#for i, review in reviews.items():
 #   reviews[i] = ' '.join([j for j in review.split() if j not in stopWords])

Połączenie reviews z danych treningowych i testowych w jedno na potrzeby zbudowania słownika

In [8]:
reviews_connected = reviews.append(test_data)

Budowanie słownika

In [9]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=1)
vectors = vectorizer.fit_transform(reviews_connected.tolist())

Podział na zbiór testowy i treningowy

In [10]:
X_train = vectors[:202645]
X_test = vectors[202645:]

Wykorzystanie LinearSVC() do klasyfikacji

In [11]:
classifier = LinearSVC()
classifier.fit(X_train, ratings)
preds = classifier.predict(X_test)

Zpis otrzymanych wartości do pliku

In [12]:
ids = test['Id']

with open('resultsSVCTfidf2.csv', 'w') as f:
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow(("Id", "Rating"))
    writer.writerows(zip(ids, preds))